1. Install needed nugets for the demo

In [1]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:Microsoft.ML.TimeSeries,*-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1 Microsoft.ML.TimeSeries, 2.0.0-preview.22313.1

2. Add global usings

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.Transforms.TimeSeries;
using System.IO;
using System.Text;

3. Create input data model

In [1]:
public class BitcoinInput
{
	[LoadColumn(1)]
	public float Open {get;set;}
}

4. Create output data model

In [1]:
public class BitcoinOutput
{
	[ColumnName("Open")]
	public float[] Open { get; set; }

    [ColumnName("Open_LB")]
    public float[] Open_LB { get; set; }

    [ColumnName("Open_UB")]
    public float[] Open_UB { get; set; }
}

5. Load the dataset and split to training and test splits

In [1]:
var mlContext = new MLContext();
var trainingData = mlContext.Data.LoadFromTextFile<BitcoinInput>("../../../ML.NET.Demo/Assets/BTC-USD.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingData, testFraction: 0.2);

6. *Optional* Display the dataset

In [1]:
dataSplit.TestSet.ToTabularDataResource().Display();

7. Train the model (Feature Engineering)

In [1]:
var model = mlContext.Forecasting.ForecastBySsa(windowSize:10, seriesLength:23, trainSize:2788, horizon:30, outputColumnName: "Open", inputColumnName: nameof(BitcoinInput.Open), confidenceLowerBoundColumn: "Open_LB", confidenceUpperBoundColumn: "Open_UB");

var trainedModel = model.Fit(dataSplit.TrainSet);

8. Save the model

In [1]:
var predictionEngine = trainedModel.CreateTimeSeriesEngine<BitcoinInput, BitcoinOutput>(mlContext);

predictionEngine.CheckPoint(mlContext, "./Forecast.zip");

9.  Load and test

In [1]:
using (var file = File.OpenRead("./Forecast.zip"))
{
    ITransformer model = mlContext.Model.Load(file, out DataViewSchema  _);
	var predictionEngineCopy = model.CreateTimeSeriesEngine<BitcoinInput, BitcoinOutput>(mlContext);
    var results = predictionEngineCopy.Predict();

    results.Display();
}

Open,Open_LB,Open_UB
"[ 39870.12, 40174.203, 40143.996, 39902.65, 39601.7, 39355.04, 39236.12, 39283.95, 39474.676, 39674.402, 39829.6, 39889.742, 39840.773, 39711.9, 39559.043, 39434.707, 39365.66, 39353.61, 39383.973, 39422.285 ... (10 more) ]","[ 38627.58, 38166.48, 37382.383, 36483.855, 35645.16, 34952.273, 34426.188, 34071.625, 33861.918, 33611.65, 33281.11, 32836.207, 32282.021, 31664.664, 31050.57, 30494.057, 30013.723, 29599.45, 29230.514, 28866.51 ... (10 more) ]","[ 41112.664, 42181.926, 42905.61, 43321.44, 43558.24, 43757.805, 44046.055, 44496.273, 45087.434, 45737.156, 46378.094, 46943.277, 47399.523, 47759.133, 48067.516, 48375.36, 48717.598, 49107.77, 49537.434, 49978.062 ... (10 more) ]"
